# Preston's Test file :)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Loading feature data

In [2]:
data_df_full = pd.read_csv('datasets/TRY_Categorical_Traits_Lookup_Table_2012_03_17_TestRelease.csv')
print('size:', data_df_full.size)
data_df_full.head()

size: 3104021


C:\Users\whitcombp\AppData\Local\Temp\ipykernel_18752\767167101.py:1: DtypeWarning: Columns (8,10,12,13,19,21,23,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df_full = pd.read_csv('datasets/TRY_Categorical_Traits_Lookup_Table_2012_03_17_TestRelease.csv')


,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,80,Abronia maritima,x,Abronia,maritima,Nyctaginaceae,Angiosperm_Eudicotyl,herb,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,139,Abutilon theophrasti,x,Abutilon,theophrasti,Malvaceae,Angiosperm_Eudicotyl,herb,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169,Acacia aneura,x,Acacia,aneura,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,197,Acacia binervata,x,Acacia,binervata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,218,Acacia burkittii,x,Acacia,burkittii,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
source_columns = data_df_full.columns.str.contains('source', case=False)
first_col = np.where(source_columns)[0][0]
last_col = len(data_df_full.columns)
df = data_df_full.drop(data_df_full.columns[list(range(first_col, last_col))], axis=1)

In [4]:
df[df['AccSpeciesName'].str.contains('magnificum', case=False)].head(10)

,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,Epiphyte,Crop,Palmoid,LeafType,LeafPhenology,PhotosyntheticPathway,Woodiness,WoodinessDetail,LeafCompoundness,NumberOfLeaflets
50407,65294,Adenocalymma magnificum,x,Adenocalymma,magnificum,Bignoniaceae,Angiosperm_Eudicotyl,NaN,NaN,NaN,...,NaN,NaN,NaN,broadleaved,NaN,NaN,NaN,NaN,NaN,NaN


## loading domes' plants data

In [5]:
domes_plants_df_full = pd.read_csv("Plants_Formatted.csv", encoding="latin-1")
domes_plants_df_full.head()

,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,Silver-Dollar Fern,Adiantum peruvianum,3,\r\n,x,x,\r\n,\r\n
1,Maidenhair Fern,Adiantum raddianum,2,\r\n,x,x,\r\n,\r\n
2,Rosy Maidenhair Fern,Adiantum ternerum 'Scutum Roseum',3,\r\n,x,x,\r\n,\r\n
3,Diamond Maidenhair Fern,Adiantum trapeziforme,6,\r\n,x,x,\r\n,\r\n
4,Living Vase Bromeliad,Aechmea 'Better than Bert',2,\r\n,x,x,\r\n,"one is potted, easily transplanted"


In [6]:
def clean(string : str) -> str:
    # full string cleaning
    string = string.lower()

    # string formatting cleaning
    substrings = string.split(' ')
    ignore = ['\'' in s for s in substrings]
    ignore.append(True) # ensures it exists in array since index() fails otherwise
    nonignored = substrings[:ignore.index(True)] # we don't care about anything after the first '
    if len(nonignored) <= 1: # can't skip first word, too short
        return nonignored[0] # remove from list for consistency
    else:
        nonignored = nonignored[1:] # skip first word (genus)
        return ' '.join(nonignored)

In [7]:
clean("Anthurium 'Big Red Bird' x A. 'Round Wan'")

'anthurium'

In [8]:
domes_plants_df_full['Shortened Scientific'] = domes_plants_df_full['Scientific Name'].apply(clean)
domes_plants_df_full['Shortened Scientific'].head()

0      peruvianum
1       raddianum
2        ternerum
3    trapeziforme
4         aechmea
Name: Shortened Scientific, dtype: object

In [9]:
df['Shortened Scientific'] = df['AccSpeciesName'].apply(clean)
df['Shortened Scientific'].head()

0       maritima
1    theophrasti
2         aneura
3      binervata
4      burkittii
Name: Shortened Scientific, dtype: object

## joining

In [10]:
joined = df.merge(domes_plants_df_full, on='Shortened Scientific')

In [11]:
print('size =', joined.shape)
joined.head()

size = (3923, 31)


,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,NumberOfLeaflets,Shortened Scientific,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,302,Acacia elata,x,Acacia,elata,Fabaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,elata,SOAP TREE,Yucca elata,2,\r\n,\r\n,X,\r\n,\r\n
1,1390,Aegilops comosa,x,Aegilops,comosa,Poaceae,Angiosperm_Monocotyl,graminoid,NaN,NaN,...,NaN,comosa,Pineapple Lily,Eucomis comosa,1,\r\n,x,x,\r\n,potted
2,1595,Aextoxicon punctatum,x,Aextoxicon,punctatum,Aextoxicaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,punctatum,Elkhorn Fern,Microsorum punctatum 'Serratum',1,\r\n,x,x,\r\n,growing on a rock
3,2221,Albizia chinensis,x,Albizia,chinensis,Fabaceae,Angiosperm_Eudicotyl,tree,NaN,NaN,...,NaN,chinensis,Jojoba,Simmondsia chinensis,3,X,\r\n,X,\r\n,Largest plant unlikely to survive but smaller ...
4,4257,Apoplanesia paniculata,x,Apoplanesia,paniculata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,NaN,NaN,...,NaN,paniculata,Orange Jasmine,Murraya paniculata,2,\r\n,x,x,\r\n,\r\n


## More cleaning!! :)

In [12]:
joined = joined.dropna(how='all')
joined = joined.fillna(0)
joined.head()

,AccSpeciesID,AccSpeciesName,IPNI/TROPICOS,Genus,SpeciesEpithet,Family,PhylogeneticGroup,PlantGrowthForm,Succulent,climber,...,NumberOfLeaflets,Shortened Scientific,Common Name,Scientific Name,Qty,Buy New & Won't Survive/Not Worth Moving,Buy New & Readily Available,Move It & Can be done by Domes staff,Move It & Requires consult - might not survive move,Notes
0,302,Acacia elata,x,Acacia,elata,Fabaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,elata,SOAP TREE,Yucca elata,2,\r\n,\r\n,X,\r\n,\r\n
1,1390,Aegilops comosa,x,Aegilops,comosa,Poaceae,Angiosperm_Monocotyl,graminoid,0,0,...,0,comosa,Pineapple Lily,Eucomis comosa,1,\r\n,x,x,\r\n,potted
2,1595,Aextoxicon punctatum,x,Aextoxicon,punctatum,Aextoxicaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,punctatum,Elkhorn Fern,Microsorum punctatum 'Serratum',1,\r\n,x,x,\r\n,growing on a rock
3,2221,Albizia chinensis,x,Albizia,chinensis,Fabaceae,Angiosperm_Eudicotyl,tree,0,0,...,0,chinensis,Jojoba,Simmondsia chinensis,3,X,\r\n,X,\r\n,Largest plant unlikely to survive but smaller ...
4,4257,Apoplanesia paniculata,x,Apoplanesia,paniculata,Fabaceae,Angiosperm_Eudicotyl,shrub/tree,0,0,...,0,paniculata,Orange Jasmine,Murraya paniculata,2,\r\n,x,x,\r\n,\r\n


# Testing New Idea!!!!
Trying to use kaggle dataset with existing pair labels to extrapolate the data to larger plant species dataset  
Process:  
 - take labeled kaggle dataset
 - expand dataset to TRY plant features
 - fit supervised learning model
 - test with larger TRY plant dataset

In [13]:
pairs_df_full = pd.read_csv('datasets/companion_plants.csv')
print('shape:', pairs_df_full.shape)
pairs_df_full.head(5)

shape: (995, 4)


,Source Node,Link,Destination Node,Source Type
0,alliums,helps,fruit trees,vegetables
1,alliums,helps,nightshades,vegetables
2,alliums,helps,tomatoes,vegetables
3,alliums,helps,capsicum,vegetables
4,alliums,helps,peppers,vegetables


In [14]:
pairs_df_full[pairs_df_full['Source Node'] == 'fruit trees'] 
# 'fruit trees' isn't a common name, so these will return empty lists when converted to scientific :(
# Meaning these links will be dropped later, as well as the 'fruit trees' in the Destination Node

,Source Node,Link,Destination Node,Source Type
757,fruit trees,helped_by,alliums,fruits
758,fruit trees,helped_by,tansy,fruits
759,fruit trees,helped_by,nasturtiums,fruits
760,fruit trees,helped_by,marigolds,fruits
761,fruit trees,helped_by,marjoram,fruits
762,fruit trees,helped_by,mustard,fruits
763,fruit trees,helped_by,delions,fruits
764,fruit trees,helped_by,borage,fruits
860,fruit trees,helped_by,lemon balm,fruits


In [15]:
### WARNING: this will take approx. 8 hours to run ###
# from EcoNameTranslator import to_species
# scientific_names_source_helps_or_hurts_destination = []
# already_looked_up_names = {} # store calculations to avoid repeating them
# unique_common_names = pairs_df_full['Source Node'].unique()
# for name in tqdm(unique_common_names, position=0, desc='Source common names'):
#     # check if name is already looked up
#     if name not in already_looked_up_names:
#         source_scientific_names = to_species([name], includeCommon=False)[name] # add all scientific names linked to source's common name
#         already_looked_up_names[name] = source_scientific_names # store to avoid recalculating
#     else:
#         source_scientific_names = already_looked_up_names[name] # get already calculated result

#     target_source_rows = pairs_df_full[pairs_df_full['Source Node'] == name] # get all rows with this name
#     for row in target_source_rows.itertuples(): # iterate over all destinations as tuple list (needs indexing) 
#         link = row[2] # Link; helps, hurts, or helped by
#         destination_node = row[3] # Destination Node; what the other plant is
#         # check if name is already looked up
#         if destination_node not in already_looked_up_names:
#             destination_scientific_names = to_species([destination_node])[destination_node] # get all scientific names of the other plant
#             already_looked_up_names[destination_node] = destination_scientific_names
#         else: 
#             destination_scientific_names = already_looked_up_names[destination_node]
#         # we want to add all pairs of source and destination scientific names to the new dataframe
#         for source_name in source_scientific_names:
#             for destination_name in destination_scientific_names:
#                 scientific_names_source_helps_or_hurts_destination.append({
#                     'Source Common' : name,
#                     'Source Scientific' : source_name,
#                     'Destination Common' : destination_node,
#                     'Destination Scientific' : destination_name,
#                     'Link' : link
#                 })
# import pickle

# with open("my_dict.pkl", "wb") as f:
#     pickle.dump(scientific_names_source_helps_or_hurts_destination, f) # MAKE SURE TO SAVE IT AFTER COMPUTE ASDFG

In [16]:
plants_common_to_scientific = pd.read_csv('datasets/plantlst.txt', delimiter=',')
plants_common_to_scientific = plants_common_to_scientific[['Scientific Name with Author', 'Common Name']]
plants_common_to_scientific['Scientific Name with Author'] = plants_common_to_scientific['Scientific Name with Author'].str.split().str[:2].str.join(sep=" ") # take first 2 words
plants_common_to_scientific = plants_common_to_scientific.dropna(ignore_index=True)
print('size of plants_common_to_scientific:', plants_common_to_scientific.shape)
def to_species(name : list) -> list: # keep formatting of EcoNameTranslator API
    name = name[0]
    filtered = plants_common_to_scientific[plants_common_to_scientific['Common Name'].str.lower().str.contains(name.lower())]
    return {name : filtered['Scientific Name with Author'].tolist()} # keep formatting of EcoNameTranslator API
to_species(['acacia'])['acacia'][:10]

size of plants_common_to_scientific: (43776, 2)


['Acacia acinacea',
 'Acacia Mill.',
 'Acacia ampliceps',
 'Acaciella angustissima',
 'Acaciella angustissima',
 'Acaciella angustissima',
 'Acacia auriculiformis',
 'Acacia confusa',
 'Acacia cultriformis',
 'Acaciella glauca']

In [17]:
# this uses the above function to query plantlst.txt instead of any API, making it much faster (but less accurate, in theory)
scientific_names_source_helps_or_hurts_destination = []
already_looked_up_names = {} # store calculations to avoid repeating them
unique_common_names = pairs_df_full['Source Node'].unique()
for name in tqdm(unique_common_names, position=0, desc='Source common names'):
    # check if name is already looked up
    if name not in already_looked_up_names:
        source_scientific_names = to_species([name])[name] # add all scientific names linked to source's common name
        already_looked_up_names[name] = source_scientific_names # store to avoid recalculating
    else:
        source_scientific_names = already_looked_up_names[name] # get already calculated result

    target_source_rows = pairs_df_full[pairs_df_full['Source Node'] == name] # get all rows with this name
    for row in target_source_rows.itertuples(): # iterate over all destinations as tuple list (needs indexing) 
        link = row[2] # Link; helps, hurts, or helped by
        destination_node = row[3] # Destination Node; what the other plant is
        # check if name is already looked up
        if destination_node not in already_looked_up_names:
            destination_scientific_names = to_species([destination_node])[destination_node] # get all scientific names of the other plant
            already_looked_up_names[destination_node] = destination_scientific_names
        else: 
            destination_scientific_names = already_looked_up_names[destination_node]
        # we want to add all pairs of source and destination scientific names to the new dataframe
        for source_name in source_scientific_names:
            for destination_name in destination_scientific_names:
                scientific_names_source_helps_or_hurts_destination.append({
                    'Source Common' : name,
                    'Source Scientific' : source_name,
                    'Destination Common' : destination_node,
                    'Destination Scientific' : destination_name,
                    'Link' : link
                })

Source common names: 100%|██████████| 97/97 [00:05<00:00, 19.31it/s]


In [18]:
hurts_help_df = pd.DataFrame(scientific_names_source_helps_or_hurts_destination)
print('hurts_help_df shape:', hurts_help_df.shape)
hurts_help_df.head()

hurts_help_df shape: (501071, 5)


,Source Common,Source Scientific,Destination Common,Destination Scientific,Link
0,asparagus,Asparagus aethiopicus,parsley,Aethusa cynapium,helps
1,asparagus,Asparagus aethiopicus,parsley,Aletes acaulis,helps
2,asparagus,Asparagus aethiopicus,parsley,Aletes anisatus,helps
3,asparagus,Asparagus aethiopicus,parsley,Aletes J.M.,helps
4,asparagus,Asparagus aethiopicus,parsley,Aletes filifolius,helps


### Next step: join hurts / help information with plant feature dataset on scientific name

In [19]:
plant_feature_data_df = pd.read_csv('datasets/GlobResp database_Atkin et al 2015_New Phytologist.csv')
print('plant feature data shape:', plant_feature_data_df.shape)
plant_feature_data_df.head()

plant feature data shape: (1137, 55)


,Country,Biome,Site,Latitude,Longitude,Altitude,Atm_Press,Responsible,Species,Family,...,Vcmax_m_25,Rdarka:Vcmax_LfT,Rdarka:Vcmax_25C_varQ10,MAT,MeanT_WarmestQtr,Annual_Precip,Precip_Warmest_Qtr,Measuring_Month_MeanT,ai_yr,pet_yr
0,Costa Rica,TrRF_lw,10?26'N_83?59'W,10.43,-83.98,105,100.07,Niinemets,Piper sancti-felicis,Piperaceae,...,NaN,NaN,NaN,26.1,27.2,3981,731,NaN,2.515,1583.0
1,Costa Rica,TrRF_lw,10?28'N_84?02'W,10.47,-84.03,140,99.66,Niinemets,Pentaclethra macroloba,Fabaceae,...,719.7,0.01,0.02,25.6,26.6,4168,750,NaN,2.658,1568.0
2,Bangladesh,TrRF_lw,24?12'N_90?9'E,24.20,90.15,21,101.07,Niinemets,Bischofia javanica,Phyllanthaceae,...,NaN,NaN,NaN,25.5,28.5,1970,736,NaN,1.344,1466.0
3,USA-PN,TeDF,40?47'N_77?53'W,40.78,-77.88,348,97.23,Niinemets,Acer saccharum,Sapindaceae,...,493.0,0.03,0.03,9.5,20.6,986,285,NaN,0.986,1000.0
4,USA-PN,TeDF,40?48'N_77?50'W,40.80,-77.83,335,97.38,Niinemets,Acer platanoides,Sapindaceae,...,1013.5,0.01,0.01,9.6,20.8,984,286,NaN,0.972,1012.0


In [20]:
joined = pd.merge(left=hurts_help_df, right=plant_feature_data_df, left_on='Source Scientific', right_on='Species')
print('merged size:', joined.shape)
joined.head()

merged size: (1793, 60)


,Source Common,Source Scientific,Destination Common,Destination Scientific,Link,Country,Biome,Site,Latitude,Longitude,...,Vcmax_m_25,Rdarka:Vcmax_LfT,Rdarka:Vcmax_25C_varQ10,MAT,MeanT_WarmestQtr,Annual_Precip,Precip_Warmest_Qtr,Measuring_Month_MeanT,ai_yr,pet_yr
0,lupin,Lupinus perennis,lettuce,Claytonia perfoliata,helps,USA-MN,TeG,Cedar Creek-OC-TeG,45.41,-93.21,...,NaN,NaN,NaN,6.3,20.4,749,314,NaN,0.835,897.0
1,lupin,Lupinus perennis,lettuce,Claytonia perfoliata,helps,USA-MN,TeG,Cedar Creek-OC-TeG,45.41,-93.21,...,NaN,NaN,NaN,6.3,20.4,749,314,NaN,0.835,897.0
2,lupin,Lupinus perennis,lettuce,Claytonia perfoliata,helps,USA-MN,TeG,Cedar Creek-OC-TeG,45.41,-93.21,...,NaN,NaN,NaN,6.3,20.4,749,314,NaN,0.835,897.0
3,lupin,Lupinus perennis,lettuce,Claytonia perfoliata,helps,USA-MN,TeG,Cedar Creek-OC-TeG,45.41,-93.21,...,NaN,NaN,NaN,6.3,20.4,749,314,NaN,0.835,897.0
4,lupin,Lupinus perennis,lettuce,Dudleya caespitosa,helps,USA-MN,TeG,Cedar Creek-OC-TeG,45.41,-93.21,...,NaN,NaN,NaN,6.3,20.4,749,314,NaN,0.835,897.0


### Cleaning time!

In [21]:
joined['PFT_LPJ'].unique() # see how many unique items exist for feasibility as categorical data

array(['TmpH', 'TrpEvBl', 'TmpEvBl'], dtype=object)

In [22]:
cleaned = joined.copy()
# NaN's
cleaned = cleaned.dropna(how='all')
cleaned = cleaned.fillna(0.0)
# convert target 'Link' feature to int
link_categories = cleaned['Link'].unique()
cat_to_int = {cat: i for i, cat in enumerate(link_categories)}
cleaned['Link'] = [cat_to_int[val] for val in cleaned['Link']]
cleaned['Link'] = cleaned['Link'].astype(int)
# filter columns
categorical_columns = ['Country', 'Biome', 'Site', 'Latitude', 'Longitude',
                       'PFT_LPJ', 'PFT_Sheffield', 'PFT_JULES', 'PFT_Wright', 'PFT_JULES2', 'PFT_Exeter' # plant functional type
]
cleaned[categorical_columns] = cleaned[categorical_columns].astype('category')
numeric_cols = cleaned.select_dtypes(include=[np.number]).columns.tolist()
columns_to_keep = categorical_columns + numeric_cols
cleaned = cleaned[columns_to_keep] # filter
# convert categorical to one-hot-encoded dummy ints
cleaned = pd.get_dummies(cleaned, columns=categorical_columns)
print('shape after cleaning:', cleaned.shape)
cleaned.head()

shape after cleaning: (1793, 76)


,Link,Altitude,Atm_Press,Aa_sat,gs_sat,Ci_sat,LfT_sat,Rdark_a,LfT_Rdark,Am_sat,...,PFT_JULES_S,PFT_Wright_AnC3H,PFT_Wright_AnEvBlS,PFT_Wright_AnEvBlT,PFT_JULES2_C3H,PFT_JULES2_EvBlT,PFT_JULES2_EvS,PFT_Exeter_BlEvTrp,PFT_Exeter_C3H,PFT_Exeter_SEv
0,0,300,97.78,20.9,0.834,0.0,25.0,1.92,25.0,321.9,...,False,True,False,False,True,False,False,False,True,False
1,0,300,97.78,20.9,0.834,0.0,25.0,1.92,25.0,321.9,...,False,True,False,False,True,False,False,False,True,False
2,0,300,97.78,20.9,0.834,0.0,25.0,1.92,25.0,321.9,...,False,True,False,False,True,False,False,False,True,False
3,0,300,97.78,20.9,0.834,0.0,25.0,1.92,25.0,321.9,...,False,True,False,False,True,False,False,False,True,False
4,0,300,97.78,20.9,0.834,0.0,25.0,1.92,25.0,321.9,...,False,True,False,False,True,False,False,False,True,False


### Training time!

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
import random
# random.seed(79) # seed for replicability

In [24]:
# standardizing numerical columns
scaler = StandardScaler()
for col in numeric_cols:
    if col != 'Link': # don't scale target int column!
        cleaned[col] = scaler.fit_transform(cleaned[col].to_numpy().reshape(-1, 1))

In [25]:
X = cleaned.drop('Link', axis=1)
y = cleaned['Link']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=79)

In [27]:
# k-fold cross validation for feature selection and best depth
# Note: this DOES NOT fully explore all feature permutations (Exploring 5 features would be C(79,5) = 24 million)
#       Full exploration is computationally impossible
#       We'll use forward and backward selection instead!
# Forward selection: (starting with no features)
# hyper parameters:
k_folds = 5
depth_range = range(1, 20)

# loop variables:
forward_best_err = -float('inf')
forward_best_depth = 0
forward_best_columns = set() # no duplicates
feature_columns = list(X.columns)
random.shuffle(feature_columns) # randomize order for fairness
feature_columns = set(feature_columns)
for col in tqdm(feature_columns): # don't include Link column; only use X columns
    for depth in depth_range:
        clf = RandomForestClassifier(max_depth=depth, random_state=79)
        experiment_X = X_train[list(forward_best_columns) + [col]]
        cv_results = cross_validate(clf, experiment_X, y_train, cv=k_folds, 
                                    scoring=('neg_mean_squared_error'))
        max_err = np.max(cv_results['test_score']) # max b/c negative
        if max_err > forward_best_err: # strictly greater than to encourage fewer features
            forward_best_err = max_err
            forward_best_depth = depth
            forward_best_columns.add(col)

print('best error:', forward_best_err)
print('best columns:', forward_best_columns)
print('best depth:', forward_best_depth)

100%|██████████| 75/75 [39:41<00:00, 31.75s/it] 

best error: -0.07806691449814127
best columns: {'PFT_Exeter_SEv', 'Aa_sat', 'Biome_TrRF_lw'}
best depth: 3


In [28]:
# Backwards selection: (starting with all features)
# hyper parameters:
k_folds = 5
depth_range = range(1, 20)

# loop variables:
backward_best_err = -float('inf')
backward_best_depth = 0
backward_best_columns = []
feature_columns = list(X.columns)
random.shuffle(feature_columns) # randomize order for fairness
feature_columns = set(feature_columns)
for col in tqdm(X.columns): # don't include Link column; only use X columns
    for depth in depth_range:
        clf = RandomForestClassifier(max_depth=depth, random_state=79)
        experiment_X = X_train[list(feature_columns)] # remove experimental column for this iteration only
        cv_results = cross_validate(clf, experiment_X, y_train, cv=k_folds, 
                                    scoring=('neg_mean_squared_error'))
        max_err = np.max(cv_results['test_score']) # max b/c negative
        if max_err >= backward_best_err: # greater or equals to encourage fewer features
            backward_best_err = max_err
            backward_best_depth = depth
            backward_best_columns = list(feature_columns) # store current best feature combination
    feature_columns.pop() # remove feature at each iteration

print('best error:', backward_best_err)
print('best columns:', backward_best_columns)
print('best depth:', backward_best_depth)

100%|██████████| 75/75 [24:25<00:00, 19.55s/it]

best error: -0.07806691449814127
best columns: ['Rdarkm_25C_varQ10', 'pet_yr', 'Rdarka_MeasMonth.T_varQ10', 'PFT_Wright_AnEvBlT', 'PFT_LPJ_TrpEvBl']
best depth: 19


In [32]:
# finding global experimental best:
if forward_best_err <= backward_best_err:
    print('best == forwards')
    best_columns = list(forward_best_columns)
    best_depth = forward_best_depth
else:
    print('best == backwards')
    best_columns = list(backward_best_columns)
    best_depth = backward_best_depth

best == forwards


### Testing time!
We'll test on both the test set, AND the full GlobResp dataset to see a more realistic comparison

In [30]:
# testing on test set
best_clf = clf = RandomForestClassifier(max_depth=best_depth, random_state=79)
cv_results = cross_validate(clf, X_test[best_columns], y_test, cv=k_folds, 
                                    scoring=('neg_mean_squared_error'))
mean_err = np.mean(cv_results['test_score'])
print('mean error on test set:', mean_err)

mean error on test set: -0.06461922596754058


# Other option: compute similarity by geographic proximity

In [31]:
from pygbif import occurrences

# Example: Get 300 occurrences of "Solanum lycopersicum" (tomato)
res = occurrences.search(
    scientificName="Aegilops comosa",
    hasCoordinate=True,
    limit=300
)

# Convert to a DataFrame
import pandas as pd
df = pd.DataFrame(res["results"])

# Keep essential geographic fields
df = df[["species", "decimalLatitude", "decimalLongitude", 
         "eventDate", "basisOfRecord", "coordinateUncertaintyInMeters"]]

print(df.head())


           species  decimalLatitude  decimalLongitude   eventDate  \
0  Aegilops comosa         37.33833          22.65500  2018-05-20   
1  Aegilops comosa         37.52389          22.63528  2018-05-18   
2  Aegilops comosa         37.32889          22.58000  2018-05-24   
3  Aegilops comosa         37.61667          22.60639  2018-05-17   
4  Aegilops comosa         37.51500          23.10167  2018-05-14   

        basisOfRecord  coordinateUncertaintyInMeters  
0  PRESERVED_SPECIMEN                            NaN  
1  PRESERVED_SPECIMEN                            NaN  
2  PRESERVED_SPECIMEN                            NaN  
3  PRESERVED_SPECIMEN                            NaN  
4  PRESERVED_SPECIMEN                            NaN  
